# Build and run DL models with TVM

官网 doc: https://tvm.apache.org/docs/how_to/compile_models/index.html

茴香豆的茴有几种写法? -- 跟 TVM 编译模型的方法一样多。

1. TVM build & run 模型，有多种 API 组合可选。粗读代码发现，一套 API 组合对应一套轮子。
2. 功能基本一样，用起来也基本一样。我感觉，掌握一套即可。
3. 我选了 `relay.build` + `tvm.contrib.graph_executor`，可以 `export_library` API 把编译后的模型保存为 `.so` 文件，方便 deploy 分发。

In [1]:
import tvm
from tvm import relay
# from tvm.ir.module import IRModule

import numpy as np

import torch
import torchvision
from torchvision import transforms

from PIL import Image

print('tvm versin: %s' % tvm.__version__)

tvm versin: 0.13.dev0


# 1. 材料准备

为了方便看 IR 和 Graph，我选了一个简单的模型：`AlexNet`。

识别的正确性，差一点。这不重要。只要，tvm 的识别结果与 pytorch 一致，即可。

In [2]:
# model list doc: https://pytorch.org/vision/main/models.html
model = torchvision.models.alexnet(weights='IMAGENET1K_V1')
# model = torchvision.models.resnet18(weights='IMAGENET1K_V1')
model = model.eval()
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [3]:
img_path = 'test-data/cat.png'
img = Image.open(img_path).resize((224, 224))

my_preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
img = my_preprocess(img)
# 新增Batch维度
img = np.expand_dims(img, 0)

In [4]:
# export to TorchScripted model, so that we can use it in TVM
img_input_name = 'input0'
img_shape = [1, 3, 224, 224]
input_shapes = [(img_input_name, img_shape)]

input_data = torch.randn(img_shape)
scripted_model = torch.jit.trace(model, input_data).eval()

跑一下 PyTorch 的分类结果作为 TVM 的标准答案

In [5]:
with torch.no_grad():
    torch_img = torch.from_numpy(img)
    output = model(torch_img)

    # Get top-1 result for PyTorch
    top1_torch = np.argmax(output.numpy())
    print('Torch top-1 id: %d' % top1_torch)

Torch top-1 id: 285


# 2. TVM build & run API

for 新手：
1. IRModule 是一个桥梁。是 TVM 的一种中间表示 (IR)。
2. 各个 DL 框架都能通过 tvm frontend 转成 IRModule。比如: pytorch, tensorflow, onnx 等。
3. 只要转 IRModule 成功，后面的 build & run API，都是一样的。来自哪个 DL 框架，没区别。

Notes:
1. mod 是一个 IRModule。
2. params 是一个 dict，存储了模型的所有权重信息。

In [6]:
# load pytorch model to IRModule using relay(tvm frontend)
mod, params = relay.frontend.from_pytorch(scripted_model, input_shapes)

# onnx
# mod, params = relay.frontend.from_onnx(onnx_model, input_shapes)
# TensorFlow
# mod, params = relay.frontend.from_tensorflow(graph_def, shape=input_shapes)

In [7]:
# some common configurations
target = 'llvm'

## 2.1 组合 1: relay.build + tvm.contrib.graph_executor

In [8]:
# compile the model
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [9]:
from tvm.contrib import graph_executor

m = graph_executor.GraphModule(lib["default"](tvm.cpu(0)))
m.set_input(img_input_name, tvm.nd.array(img.astype('float32')))
m.run()
tvm_output = m.get_output(0)

top1_tvm_1 = np.argmax(tvm_output.numpy()[0])
print("TVM top-1 id: %s" % top1_tvm_1)

TVM top-1 id: 285


## 2.2 组合 2: build_module + evaluate

In [10]:
with tvm.transform.PassContext(opt_level=3):
    intrp = relay.build_module.create_executor("graph", mod, tvm.cpu(0), target)

In [11]:
tvm_output2 = intrp.evaluate()(tvm.nd.array(img.astype('float32')), **params)

top1_tvm_2 = np.argmax(tvm_output2.numpy()[0])
print("TVM top-1 id: %s" % top1_tvm_2)

TVM top-1 id: 285


## 2.3 组合 3: relax.build + vm

没跑通 run

In [12]:
from tvm import relax

mod_with_params = relax.transform.BindParams("main", params)(mod)
ex = relax.build(mod_with_params, target=target)

# ex = relax.build(mod, target=target)

In [13]:
vm = relax.VirtualMachine(ex, tvm.cpu())

nd_res = vm['main'](tvm.nd.array(img.astype('float32')))
top1_tvm_3 = np.argmax(nd_res.numpy()[0])
print("TVM top-1 id: %s" % top1_tvm_2)

AttributeError: Module has no function 'main'